In [8]:
import pandas as pd
from txtai.embeddings import Embeddings
import re
import unicodedata
import string

def remove_punctuation(text:str) -> str:
    return ''.join([c for c in text if c not in string.punctuation])

def remove_accented_chars(text: str) -> str:
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

def remove_numbers(text: str) -> str:
    pattern = r'[^a-zA-z.,!?/:;\"\'\s]' 
    return re.sub(pattern, '', text)

def remove_special_characters(text: str) -> str:
    pat = r'[^a-zA-z0-9.,!?/:;\"\'\s]' 
    return re.sub(pat, '', text)

## Combine Goals and Objectives (with titles) into one paragraph and make comparison

In [9]:
go = pd.read_csv("./data/processed/goals-objectives-combined.csv", index_col="goal_id")
go.sort_index(inplace=True)
go.reset_index(inplace=True)
go

,goal_id,goal_name,goal_objective
0,0,Innovate for Competitive Advantage,Innovation is a key element of future readines...
1,1,Optimize for Efficiencies and Improved Capability,Delivering IT capabilities with greater effici...
2,2,Evolve Cybersecurity for an Agile and Resilien...,The scope pace and sophistication of malicious...
3,3,Cultivate Talent for a Ready Digital Workforce,Competition for high quality experienced digit...


In [10]:
embeddings = Embeddings({"path": "sentence-transformers/nli-mpnet-base-v2"})
embeddings.index([(uid, text, None) for uid, text in enumerate(go.goal_objective.tolist())])

In [11]:
results = pd.DataFrame([(query_id, query, go.iloc[embeddings.search(query, 1)[0][0]]['goal_name'], query_description) for _, query_id, query, query_description in go.to_records()])
results.columns = ["query_id", "query", "resul_goalname", "query_description"]
for _, query_id, query, query_description in go.to_records():
    print(f"{query_id} | {query} | {go.iloc[embeddings.search(query, 1)[0][0]]['goal_name']}")
results.to_csv("./results/goal-objectives-combined-results.csv", index=False)

0 | Innovate for Competitive Advantage | Innovate for Competitive Advantage
1 | Optimize for Efficiencies and Improved Capability | Optimize for Efficiencies and Improved Capability
2 | Evolve Cybersecurity for an Agile and Resilient Defense Posture | Evolve Cybersecurity for an Agile and Resilient Defense Posture
3 | Cultivate Talent for a Ready Digital Workforce | Cultivate Talent for a Ready Digital Workforce


In [12]:
survey = pd.read_excel("./resources/JCAT Export NIPR (APR-15-2022).xlsx")
capabilities = pd.DataFrame(survey[survey.columns[-4]])
capabilities.columns = ["capability_description"]
capabilities["capability_description"] = capabilities["capability_description"].apply(lambda x: remove_special_characters(x))
capabilities["capability_description"] = capabilities["capability_description"].apply(lambda x: remove_accented_chars(x))
capabilities.head()

,capability_description
0,"GDI/ACEM is a cloud based, multi enclave SIPR/..."
1,The NorthStar Information Dominance Platform T...
2,Bodhi is NRO's Global Situational Awareness pl...
3,Maven Broad Area Surveillance Targeting BAST\...
4,High Accuracy Detection and Exploitation Syste...


In [ ]:
results = pd.DataFrame([(capability, *go.loc[embeddings.search(capability, 1)[0][0], ['goal_name', 'goal_objective']].values.tolist()) for _, capability in capabilities.to_records()])
results.columns = ["capability_query", "goal_name", "goal_objective"]
results.head()

In [ ]:
results.to_csv("./results/survey-alignment.csv", index=False)